In [3]:
import requests
import json
from app.utils.audio.tts.audio_synthesis import translate_text_to_audio
from app.utils.audio.asr.translate_audio import transcribe_audio

def query_llama_with_ollama(system_prompt, user_prompt):
    """
    Sends a prompt to LLaMA 3.2 via Ollama and streams the response.
    """
    url = "http://localhost:11434/api/generate"
    full_prompt = f"{system_prompt}\n\n{user_prompt}"
    payload = {
        "model": "gemma2",
        "prompt": full_prompt,
        "stream": True
    }

    try:
        with requests.post(url, json=payload, stream=True) as response:
            if response.status_code == 200:
                for line in response.iter_lines():
                    if line:
                        decoded_line = line.decode('utf-8')
                        json_line = json.loads(decoded_line)
                        if 'response' in json_line:
                            yield json_line['response']
            else:
                print(f"Error: {response.status_code} - {response.text}")
                yield f"Error: Failed to get response from LLaMA. Status code: {response.status_code}"

    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        yield "Error: An error occurred while making the request."

def process_pipeline(file_path: str, file_id: str):
    try:
        transcribed_audio = transcribe_audio(file_path)
        if not transcribed_audio:
            raise Exception("Error transcribing audio")
        print(f"Transcribed audio: {transcribed_audio}")
        system_prompt = """You are a storytelling AI assistant that writes stories in Urdu script.
        You need to write a creative story in Urdu based on the topic and theme provided by the user.
        Make the story interesting and fluent, and do not use any words outside of the Urdu script.
        Limit the story to a maximum length of 200 words.
        Always start the story with a proper introduction.
        Never end the story abruptly, and always conclude it logically.
        Absolutely never include a word that is not in Urdu font."""

        theme = "horror"
        topic = "  [ ایک شرارتی بلی] "

        user_prompt = f"""تھییم: [{theme}]
        موضوع: [{transcribed_audio}]
        اب اردو رسم الخط میں کہانی لکھیں:"""

        story = ""
        for response in query_llama_with_ollama(system_prompt, user_prompt):
            story += response

        audio_file_path = translate_text_to_audio(story, file_id)

        if not audio_file_path:
            raise Exception("Error translating text to audio")
        print(f"Audio file path: {audio_file_path}")

        response = {
            "status": "success",
            "message": "Pipeline processed successfully",
            "file_id": file_id,
            # URL to download the audio file Add the server URL before this in the frontend
            "audio_url": f"/get-audio/{file_id}",
            "story": story,
        }

        return response

    except Exception as e:
        print(f"Error processing pipeline: {e}")
        response = {
            "status": "error",
            "message": "Error processing pipeline",
            "error": str(e)
        }
        return response

process_pipeline("test.ogg", "dummy3")

Device set to use cuda:0
d:\Project\Udru-Storytelling--ASR-to-TTS\backend\.venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=translate, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=translate.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcribed audio:  ایک لڑکا جس کے نام دولا تھا اور وہ مرگیا تھا
Audio file path: synthesized_audio/dummy3.wav


{'status': 'success',
 'message': 'Pipeline processed successfully',
 'file_id': 'dummy3',
 'audio_url': '/get-audio/dummy3',
 'story': 'دولا نامی ایک بچہ بہت اچھا اور شُرِر تھا۔  ہر کسی کی دل میں جگہ رکھتا تھا۔ مگر ایک دن، اس کے جسم کو ایک غیر معمولی بیماری نے آگیا جس سے وہ بیمار ہو گیا۔\n\nآشپز خانے میں رات گھڑی کے وقت، دولا کی ماں نے اُس کی آنکھوں میں ایک غیر عادی چمک دیکھی۔  وہ بچہ کھڑا ہے اور کچھ کہنے لگا، مگر اس کی آواز ایک نرالے طرز پر تھی جو ان کی سمجھ سے باہر تھا۔\n\nدولا کے جسم کا رنگ گھٹ گیا اور وہ کمزور ہو گیا۔ ماں نے اپنے بیٹے کو محبت سے گھیر لیا، لیکن دولا اب بھی کچھ کہہ رہا تھا۔ وہ "میں چلا جاؤں گا" بول رہا تھا۔  \n\nدہلیز پر ایک غریب اور تاریک ڈھال نظر آئی، جیسے دولا کی روح ان سے جدا ہو رہی ہے۔ \n\n\n'}